You just got hired as the first and only data practitioner at a small business experiencing exponential growth. The company needs more structured processes, guidelines, and standards. Your first mission is to structure the human resources data. The data is currently scattered across teams and files and comes in various formats: Excel files, CSVs, JSON files...

You'll work with the following data in the `datasets` folder:
- __Office addresses__ are currently saved in `office_addresses.csv`. If the value for office is `NaN`, then the employee is remote. 
- __Employee addresses__ are saved on the first tab of `employee_information.xlsx`.
- __Employee emergency contacts__ are saved on the second tab of `employee_information.xlsx`; this tab is called `emergency_contacts`. However, this sheet was edited at some point, and the headers were removed! The HR manager let you know that they should be: `employee_id`, `last_name`, `first_name`, `emergency_contact`, `emergency_contact_number`, and `relationship`.
- __Employee roles, teams, and salaries__ have been exported from the company's human resources management system into a JSON file titled `employee_roles.json`. Here are the first few lines of that file:
```
{"A2R5H9":
  {
    "title": "CEO",
    "monthly_salary": "$4500",
    "team": "Leadership"
  },
 ...
}

In [54]:
import pandas as pd
# Read in dataframes
office_addresses = pd.read_csv('datasets/office_addresses.csv')
employee_addresses = pd.read_excel('datasets/employee_information.xlsx', sheet_name = 0)
employee_emergency_contacts = pd.read_excel('datasets/employee_information.xlsx', sheet_name = 1, header = None, names = ['employee_id', 'last_name', 'first_name', 'emergency_contact', 'emergency_contact_number', 'relationship'])
employee_roles = pd.read_json('datasets/employee_roles.json', orient= 'index')

In [55]:
# Explore dfs 
office_addresses

,office,office_country,office_city,office_street,office_street_number
0,Leuven Office,BE,Leuven,Martelarenlaan,38
1,ESB Office,US,New York City,Fifth Avenue,350
2,WeWork Office,GB,London,Old Street,207


In [56]:
employee_addresses['employee_city'] = employee_addresses['employee_city'].str.replace('New-York','New York City')
employee_addresses

,employee_id,employee_last_name,employee_first_name,employee_country,employee_city,employee_street,employee_street_number
0,A2R5H9,Hunman,Jax,BE,Leuven,Grote Markt,9
1,H8K0L6,Siff,Tara,GB,London,Baker Street,221
2,G4R7V0,Sagal,Gemma,US,New York City,Perry Street,66
3,M1Z7U9,Coates,Tig,FR,Paris,Rue de l'Université,7


In [57]:
employee_emergency_contacts

,employee_id,last_name,first_name,emergency_contact,emergency_contact_number,relationship
0,A2R5H9,Hunman,Jax,Opie Hurst,+32-456-5556-84,Brother
1,H8K0L6,Siff,Tara,Wendy de Matteo,+44-020-5554-333,Sister
2,G4R7V0,Sagal,Gemma,John Newmark,+1-202-555-194,Husband
3,M1Z7U9,Coates,Tig,Venus Noone,+1-202-555-0130,Wife


In [58]:
employee_roles

,title,monthly_salary,team
A2R5H9,CEO,$4500,Leadership
H8K0L6,CFO,$4500,Leadership
G4R7V0,Business Developer,$3000,Sales
M1Z7U9,Office Manager,$2000,People Operations


In [59]:
# Merge df into one called employees_final
employee_final = employee_addresses.merge(employee_emergency_contacts, on='employee_id')
employee_final = employee_final[['employee_id','first_name','last_name','employee_country','employee_city', 'employee_street', 'employee_street_number', 'emergency_contact','emergency_contact_number','relationship']]
employee_final.set_index('employee_id',inplace=True)
employee_final = employee_final.merge(employee_roles, left_index=True, right_index=True)
employee_final.reset_index(inplace=True)
employee_final = employee_final.merge(office_addresses, how='left', left_on=['employee_country','employee_city'], right_on=['office_country','office_city'])
employee_final

,index,first_name,last_name,employee_country,employee_city,employee_street,employee_street_number,emergency_contact,emergency_contact_number,relationship,title,monthly_salary,team,office,office_country,office_city,office_street,office_street_number
0,A2R5H9,Jax,Hunman,BE,Leuven,Grote Markt,9,Opie Hurst,+32-456-5556-84,Brother,CEO,$4500,Leadership,Leuven Office,BE,Leuven,Martelarenlaan,38.0
1,H8K0L6,Tara,Siff,GB,London,Baker Street,221,Wendy de Matteo,+44-020-5554-333,Sister,CFO,$4500,Leadership,WeWork Office,GB,London,Old Street,207.0
2,G4R7V0,Gemma,Sagal,US,New York City,Perry Street,66,John Newmark,+1-202-555-194,Husband,Business Developer,$3000,Sales,ESB Office,US,New York City,Fifth Avenue,350.0
3,M1Z7U9,Tig,Coates,FR,Paris,Rue de l'Université,7,Venus Noone,+1-202-555-0130,Wife,Office Manager,$2000,People Operations,NaN,NaN,NaN,NaN,NaN


In [60]:
# Fill nulls in office columns for 'Remote'
for col in ['office', 'office_country', 'office_city', 'office_street', 'office_street_number']:
    employee_final[col].fillna('Remote', inplace=True)
employee_final

,index,first_name,last_name,employee_country,employee_city,employee_street,employee_street_number,emergency_contact,emergency_contact_number,relationship,title,monthly_salary,team,office,office_country,office_city,office_street,office_street_number
0,A2R5H9,Jax,Hunman,BE,Leuven,Grote Markt,9,Opie Hurst,+32-456-5556-84,Brother,CEO,$4500,Leadership,Leuven Office,BE,Leuven,Martelarenlaan,38.0
1,H8K0L6,Tara,Siff,GB,London,Baker Street,221,Wendy de Matteo,+44-020-5554-333,Sister,CFO,$4500,Leadership,WeWork Office,GB,London,Old Street,207.0
2,G4R7V0,Gemma,Sagal,US,New York City,Perry Street,66,John Newmark,+1-202-555-194,Husband,Business Developer,$3000,Sales,ESB Office,US,New York City,Fifth Avenue,350.0
3,M1Z7U9,Tig,Coates,FR,Paris,Rue de l'Université,7,Venus Noone,+1-202-555-0130,Wife,Office Manager,$2000,People Operations,Remote,Remote,Remote,Remote,Remote


In [61]:
# Set index and select the right columns
employee_final.set_index('index',inplace=True)
employee_final = employee_final[['first_name', 'last_name', 'employee_country', 'employee_city', 'employee_street', 'employee_street_number', 'emergency_contact', 'emergency_contact_number', 'relationship', 'monthly_salary', 'team', 'title', 'office', 'office_country', 'office_city', 'office_street', 'office_street_number']]
employee_final

,first_name,last_name,employee_country,employee_city,employee_street,employee_street_number,emergency_contact,emergency_contact_number,relationship,monthly_salary,team,title,office,office_country,office_city,office_street,office_street_number
index,,,,,,,,,,,,,,,,,
A2R5H9,Jax,Hunman,BE,Leuven,Grote Markt,9,Opie Hurst,+32-456-5556-84,Brother,$4500,Leadership,CEO,Leuven Office,BE,Leuven,Martelarenlaan,38.0
H8K0L6,Tara,Siff,GB,London,Baker Street,221,Wendy de Matteo,+44-020-5554-333,Sister,$4500,Leadership,CFO,WeWork Office,GB,London,Old Street,207.0
G4R7V0,Gemma,Sagal,US,New York City,Perry Street,66,John Newmark,+1-202-555-194,Husband,$3000,Sales,Business Developer,ESB Office,US,New York City,Fifth Avenue,350.0
M1Z7U9,Tig,Coates,FR,Paris,Rue de l'Université,7,Venus Noone,+1-202-555-0130,Wife,$2000,People Operations,Office Manager,Remote,Remote,Remote,Remote,Remote
